# Church Encoding
Creating the basic functionality in Lambda Calculus using a function-based encoding scheme developed initially by Alonzo Church and later refined by Haskell Curry and others. 

The initial version of $\lambda$-Calculus does not have Boolean types, numbers, loops, Boolean operations, if statements or even a concept of zero. The only thing it has is a pure function, a function that has no side effects and only takes a single (unary) input and produces an output. No changes are made else where, such as a terminal output etc. All elements must be built from the ground up by encoding everything into unary, pure functions. 

## Boolean Algebra
We will begin by defining the concepts of true and false and then build the necessary minimal components for Boolean algebra.

In [2]:
#from inspect import signature #python 3.3+

T = lambda t: lambda f: t
F = lambda t: lambda f: f

#use annotations to tell what function is returned in verifying the result
T.__annotations__['T'] = True
F.__annotations__['F'] = False

#also can tell by converting to Python bool type, cheating a bit though
to_bool = lambda boolean: boolean(True)(False)

To verify that this is consistent, let us build AND, OR and NOT functions. The NOT should take a Boolean and toggle its value.

In [3]:
NOT = lambda boolean: boolean(F)(T)

In [4]:
NOT(F).__annotations__

{'T': True}

In [5]:
to_bool(NOT(F))

True

The AND operation should only return True if both arguments are True.

In [6]:
AND = lambda bool1: lambda bool2: bool1(bool2)(F)

In [7]:
AND(T)(F).__annotations__

{'F': False}

The OR operation should only return True if both or either of the arguments are True.

In [8]:
OR = lambda bool1: lambda bool2: bool1(T)(bool2)

In [9]:
OR(T)(F).__annotations__

{'T': True}

To do conditional statements or arithmetic we need numbers

## Numerals
We only have functions to work with, so we represent numbers as the number of times we apply a function to an argument. To represent zero, we apply a function zero times and to represent one we apply the function one time to the argument. The form of the function is not important right now, so using the identity gives us the numbers we want, but using other functions can be useful too.

In [10]:
#identity
I = lambda x: x

#numerals
N0 = lambda f: lambda x: x
N1 = lambda f: lambda x: f(x)
N2 = lambda f: lambda x: f(f(x))
N3 = lambda f: lambda x: f(f(f(x)))
N4 = lambda f: lambda x: f(f(f(f(x))))

In [11]:
#use annotations to tell what function is returned in verifying the result
I.__annotations__['I'] = "I"
N0.__annotations__['0'] = 0
N1.__annotations__['1'] = 1
N2.__annotations__['2'] = 2
N3.__annotations__['3'] = 3
N4.__annotations__['4'] = 4

#also by converting to Python int type by counting function calls
#we pass the increment function into the numeral to count the calls
to_int = lambda n: n(lambda integer: integer + 1)(0)

Now we need to travel between the numbers by count up via succession. To count we need to apply an additional $f$ to our expression for the numeral in question.

In [12]:
S = lambda n: lambda f: lambda x: f(n(f)(x))

In [13]:
#use annotations to tell what function is returned in verifying the result
S.__annotations__['S'] = "Successor"

In [14]:
S(N2).__annotations__

{}

In [15]:
to_int(S(N2))

3

Now we have numbers we should be able to do addition and multiplication. Addition to $n$ is simply taking the successor function and instead of applying the additional function $f$ one time to get next numeral, we apply it $m$ times to get $n+m$.

In [16]:
ADD = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))

In [17]:
to_int(ADD(N2)(N3))

5

We can re-use the successor function to compute the additional of two numerals $n$ and $m$ also nas follows by counting up the number of successors by $m$

In [18]:
ADDS = lambda m: lambda n: lambda f: lambda x: m(S)(n)(f)(x)

In [19]:
to_int(ADDS(N3)(N4))

7

To multiply, we need to apply the $m$ number of applications of the function $f$ to the numeral $n$. This is equivalent to composition of the numerals $n$ and $m$, and so the MUL function can be writen as the composition combinator.

In [20]:
MUL = lambda m: lambda n: lambda f: lambda x: m(n(f))(x)

In [21]:
to_int(MUL(N2)(N3))

6

In [22]:
MUL = lambda m: lambda n: lambda f: m(n(f))

In [23]:
to_int(MUL(N2)(N3))

6

We can extend the idea of repeated addition by repeated multiplication to develop the exponentiation. 

In [24]:
POW = lambda m: lambda n: lambda f: lambda x: S(n)(m(f))(x)

In [25]:
to_int(POW(N3)(N2))

9

Now we can do some conditional statements

## Conditional Statements
We can do simple checks like if a number is zero, noting that the number zero is actually the identity function. We write the expression so that if we apply a function at all (say $n$ times if $n>0$) it is always false, then when $n=0$, we wont apply this function and it can be true.

In [26]:
Z = lambda x: x(lambda x: F)(T)

In [27]:
Z(N2).__annotations__

{'F': False}